# I thought we're done. No we are not.

Feature extraction part 2!

Of course, we import stuff again and then pickle in

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import re
import scipy.stats as stats
import datetime as dt


sns.set_style('darkgrid')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [ ]:
mbti = pd.read_pickle('mbti_ver1.pickle')
mbti.head()

In [ ]:
#Get median for number of characters/words per post in each row

mbti['med_char'] = mbti['words_only'].apply(lambda x: np.median([len(i) for i in x]))
mbti['med_word'] = mbti['words_only'].apply(lambda x: np.median([len(i.split()) for i in x]))
mbti.head()

In [ ]:
#Spice things up a little by splitting to individual types
mbti['is_E'] = mbti['type'].apply(lambda x: 1 if x[0] == 'E' else 0)
mbti['is_S'] = mbti['type'].apply(lambda x: 1 if x[1] == 'S' else 0)
mbti['is_T'] = mbti['type'].apply(lambda x: 1 if x[2] == 'T' else 0)
mbti['is_J'] = mbti['type'].apply(lambda x: 1 if x[3] == 'J' else 0)

#Yes this is the direction I am gonna go.

In [ ]:
mbti['word_char_ratio'] = mbti['med_char'] / mbti['med_word']

In [ ]:
mbti.columns

In [ ]:
mbti.head()

In [ ]:
#Look at number by descending count per type
mbti[['type','emoticon_count']].sort_values('emoticon_count', ascending=False).head(10)
#But then again this can be skewed towards higher participation, so...

In [ ]:
#We look at averages below
#By eyeballing we can see that feelers tend to use more emoticons on average than thinkers with the exception of ISTJ
#The rest are more uhh uniform
mbti.groupby('type')['emoticon_count'].apply(np.mean).sort_values(ascending=False)

In [ ]:
#If we look at 'is_T' instead (1 = T, 0 = F)
plt.figure(figsize=(6,6))
plt.subplot(221)
E = mbti.groupby('is_E')['emoticon_count'].apply(np.mean).sort_values(ascending=False)
plt.bar(E.index, E, width=0.3, tick_label=['I','E'], color='yellow')
plt.ylabel("n_emoticon")
plt.title("Average emoticon count of Extraverted (E) vs Introverted (I) types")

plt.subplot(222)
S = mbti.groupby('is_S')['emoticon_count'].apply(np.mean).sort_values(ascending=False)
plt.bar(S.index, S, width=0.3, tick_label=['N','S'], color='red')
plt.ylabel("n_emoticon")
plt.title("Average emoticon count of Sensing (S) vs Intiutive (N) types")

plt.subplot(223)
T = mbti.groupby('is_T')['emoticon_count'].apply(np.mean).sort_values(ascending=False)
plt.bar(T.index, T, width=0.3, tick_label=['F','T'])
plt.ylabel("n_emoticon")
plt.title("Average emoticon count of Thinking (T) vs Feeling (F) types")

plt.subplot(224)
J = mbti.groupby('is_J')['emoticon_count'].apply(np.mean).sort_values(ascending=False)
plt.bar(J.index, J, width=0.3, tick_label=['P','J'], color='green')
plt.ylabel("n_emoticon")
plt.title("Average emoticon count of Judging (J) vs Perceptive (P) types")

plt.subplots_adjust(left=0, bottom=1, right=1.1, top=2, wspace=2, hspace=0.3)

In [ ]:
#Create function heh
def compareplot(what):
    plt.figure(figsize=(6,6))
    plt.subplot(221)
    E = mbti.groupby('is_E')[what].apply(np.mean).sort_values(ascending=False)
    plt.bar(E.index, E, width=0.3, tick_label=['I','E'], color='yellow')
    plt.ylabel(what)
    plt.title("Average "+what+" of Extraverted (E) vs Introverted (I) types")

    plt.subplot(222)
    S = mbti.groupby('is_S')[what].apply(np.mean).sort_values(ascending=False)
    plt.bar(S.index, S, width=0.3, tick_label=['N','S'], color='red')
    plt.ylabel(what)
    plt.title("Average "+what+" of Sensing (S) vs Intiutive (N) types")

    plt.subplot(223)
    T = mbti.groupby('is_T')[what].apply(np.mean).sort_values(ascending=False)
    plt.bar(T.index, T, width=0.3, tick_label=['F','T'])
    plt.ylabel(what)
    plt.title("Average "+what+" of Thinking (T) vs Feeling (F) types")

    plt.subplot(224)
    J = mbti.groupby('is_J')[what].apply(np.mean).sort_values(ascending=False)
    plt.bar(J.index, J, width=0.3, tick_label=['P','J'], color='green')
    plt.ylabel(what)
    plt.title("Average "+what+" of Judging (J) vs Perceptive (P) types")

    plt.subplots_adjust(left=0, bottom=1, right=1.1, top=2, wspace=2, hspace=0.3)

In [ ]:
mbti.columns

In [ ]:
compareplot('word_char_ratio')

In [ ]:
colli = ['n_video','n_links','n_image','emoticon_count','mention_count','hashtag_count','mbti_ref_count',
       'ennea_count', 'bracket_count', 'dots_count', 'n_char', 'n_word','n_action', 'n_caps','n_caps_char',
       'word_cap_ratio','char_cap_ratio','med_char','med_word','is_E','is_S','is_T','is_J']
mbti_num = mbti[colli]
mbti_num.head()

In [ ]:
sns.pairplot(mbti_num[['is_E','emoticon_count','mention_count','hashtag_count','mbti_ref_count',
       'ennea_count', 'bracket_count', 'dots_count']], hue='is_E')

In [ ]:
sns.pairplot(mbti_num[['is_E','n_char', 'n_word','n_action', 'n_caps','n_caps_char',
       'word_cap_ratio','char_cap_ratio','med_char','med_word']], hue='is_E')

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(mbti_num[['is_E','n_char', 'n_word','n_action', 'n_caps','n_caps_char',
       'word_cap_ratio','char_cap_ratio','med_char','med_word']].corr(), annot=True)

In [ ]:
y = mbti_num['is_E']
X = mbti_num[['n_video','n_links','n_image','emoticon_count','mention_count','hashtag_count','mbti_ref_count',
       'ennea_count', 'bracket_count', 'dots_count', 'n_char', 'n_word','n_action', 'n_caps','n_caps_char',
       'word_cap_ratio','char_cap_ratio','med_char','med_word']]

I would like to first try out TPOT here (Later I will perform some magic but hey)

See how the performance will be like

In [ ]:
# from tpot import TPOTClassifier
# from sklearn.model_selection import train_test_split

# y = mbti_num['is_E']
# X = mbti_num[['n_video','n_links','n_image','emoticon_count','mention_count','hashtag_count','mbti_ref_count',
#        'ennea_count', 'bracket_count', 'dots_count', 'n_char', 'n_word','n_action', 'n_caps','n_caps_char',
#        'word_cap_ratio','char_cap_ratio','med_char','med_word']]
# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)

# tpot = TPOTClassifier(generations=10, population_size=30, verbosity=2, scoring='roc_auc')
# tpot.fit(X_train, y_train)
# print(tpot.score(X_test, y_test))
# tpot.export('tpot_E_try.py')

In [ ]:
# y = mbti_num['is_S']
# X = mbti_num[['n_video','n_links','n_image','emoticon_count','mention_count','hashtag_count','mbti_ref_count',
#        'ennea_count', 'bracket_count', 'dots_count', 'n_char', 'n_word','n_action', 'n_caps','n_caps_char',
#        'word_cap_ratio','char_cap_ratio','med_char','med_word']]
# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)

# tpot = TPOTClassifier(generations=10, population_size=30, verbosity=2, scoring='roc_auc')
# tpot.fit(X_train, y_train)
# print(tpot.score(X_test, y_test))
# tpot.export('tpot_S_try.py')

In [ ]:
# y = mbti_num['is_T']
# X = mbti_num[['n_video','n_links','n_image','emoticon_count','mention_count','hashtag_count','mbti_ref_count',
#        'ennea_count', 'bracket_count', 'dots_count', 'n_char', 'n_word','n_action', 'n_caps','n_caps_char',
#        'word_cap_ratio','char_cap_ratio','med_char','med_word']]
# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)

# tpot = TPOTClassifier(generations=10, population_size=30, verbosity=2, scoring='roc_auc')
# tpot.fit(X_train, y_train)
# print(tpot.score(X_test, y_test))
# tpot.export('tpot_T_try.py')

In [ ]:
# y = mbti_num['is_J']
# X = mbti_num[['n_video','n_links','n_image','emoticon_count','mention_count','hashtag_count','mbti_ref_count',
#        'ennea_count', 'bracket_count', 'dots_count', 'n_char', 'n_word','n_action', 'n_caps','n_caps_char',
#        'word_cap_ratio','char_cap_ratio','med_char','med_word']]
# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)

# tpot = TPOTClassifier(generations=10, population_size=30, verbosity=2, scoring='roc_auc')
# tpot.fit(X_train, y_train)
# print(tpot.score(X_test, y_test))
# tpot.export('tpot_J_try.py')

Work to do:

- NLP stuff (Bag of words, topic thingy etc)
- Look at the current numerical data to look at how good/bad the classification would be
- I'll probably hibernate to go watch some NLP lectures, hopefully can wake up in time to perform some fantastic magic

In [ ]:
mbti['words_refine'] = mbti['words_only'].apply(lambda x:[line.decode('utf-8') for line in x])

In [ ]:
import string
def bigchunk(wordie):
    #wordie = mbti['words_only'][0]
    wordchunk = ' '.join(wordie)
    nopunclist = [char for char in wordchunk if char not in string.punctuation]
    nopuncchunk = ''.join(nopunclist)
    return nopuncchunk

mbti['words_only'].apply(bigchunk)[2]
#Reference \xe2\x80\x93

Ah but what about non-english posts? They might be hidden inside!

Dig em out

In [ ]:
def notenglish(y):
    chunk = bigchunk(y)
    if re.search('[^a-zA-Z\d\s]', chunk):
        unilist = re.findall('[^a-zA-Z\d\s]+', chunk)
        print unilist
    else:
        print None
#         chunklist = chunk.split(' ')
#         newchunk = list(chunklist)
#         anotherlist=[]
#         for uni in unilist:
#             try:
#                 newchunk.remove(uni)
#             except:
#                 anotherlist.append(uni)
#         newlist = ' '.join(newchunk)
#         if anotherlist != []:
#             for ano in anotherlist:
#                 newlist = newlist.replace(ano, ' ')
#         return newlist
#     else:
#         return chunk
    
mbti['words_only'].apply(notenglish)

Chunk shall be used for tfidf later. Instead of running the model as is, I will split the model into 4 according to the type, so as to better extract the most significant significant word frequency relative to the other type.

As for other variables like counts and all that they are quite constant so I can extract them now while I still have the chance since they are not dependent on other posts.

*update*

Actually scikit learn stuffs like CountVec and Tfidf will self optimize one hahah so wth did I just do like waste my time lol

### Let's do POS!

Wait what is POS

Oh im glad you asked

POS is not Point of Sale arh! It means Parts of Speech. What I am going to do here is to extract out the type of words within a bunch of words and stuff to identify noun, adjective, punctuation etc. 

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

#Takes a long time to run!
mbti['tagged_words'] = mbti['words_only'].apply(
    lambda x: [nltk.pos_tag(word_tokenize(line.decode('utf-8', errors='replace'))) for line in x])
mbti['tagged_words'][0]

In [13]:
#From the 1st X rows, look at the type of words in each tag
#Also takes a long time! I used a for loop -.-

from pprint import pprint
wordcatdict={}
for pig in mbti[mbti['POS_LS_mean'] != 0]['tagged_words']:
    for line in pig:
        for x in line:
            if x[1] in wordcatdict.keys():         #Check for existing tag
                if x[0] in wordcatdict[x[1]]:      #Check for existing word
                    continue
                else:
                    wordcatdict[x[1]].append(x[0]) #Add 2nd or more word
            else:
                wordcatdict[x[1]] = [x[0]]         #Add new tag with new word as list
            
pprint(wordcatdict)
#Its not perfect, but hey
#Remember the purpose - it is not to exactly identify the right kind of words, but to extract features
#So its ok!

#Lemme iterate this
#I actually do not care much about the integrity of the features
#Most importantly is that it can consistently perform as a good model

{'#': [u'#'],
 '$': [u'$', u'Answer', u'Helios', u'Mar', u'/', u'vibe', u'Type', u'xSFP'],
 "''": [u"'",
        u'Wait',
        u"''",
        u"'Fine",
        u'Entp',
        u"'need",
        u'INFP',
        u'ROCK',
        u'Figure',
        u"'of",
        u"'weird"],
 '(': [u'(', u'{'],
 ')': [u')', u'}'],
 ',': [u','],
 '.': [u'?', u'.', u'!'],
 ':': [u'-', u':', u'...', u';', u'--'],
 'CC': [u'and',
        u'but',
        u'\U0001f602',
        u'or',
        u'And',
        u'&',
        u'Otherwise',
        u'But',
        u'Plus',
        u'nor',
        u'Whether',
        u'minus',
        u'Or',
        u'either',
        u'Pore',
        u'Either',
        u'So',
        u'deadmau5',
        u'Foxcatcher',
        u'nunu',
        u'ninja',
        u'No',
        u'look',
        u'plus',
        u'neither',
        u'Never',
        u'Neither',
        u']',
        u'yeah',
        u'Calculus',
        u'_N_P',
        u'yet',
        u'Yet',
        u'OR',
    

        u'Though',
        u'paths',
        u'under',
        u'unless',
        u'within',
        u'whether',
        u'towards',
        u'<',
        u'Around',
        u'Been',
        u'trough',
        u'On',
        u'Once',
        u'profile',
        u'About',
        u'Wow',
        u'besides',
        u'near',
        u'throughout',
        u'not/1',
        u'Maybe',
        u'toward',
        u'above',
        u'per',
        u'Fuck',
        u'o',
        u'why..',
        u'vibes',
        u'-love',
        u'excuse',
        u'down',
        u'oh',
        u'unlike',
        u'Out',
        u'Like',
        u'during',
        u'inside',
        u'Whats',
        u'up',
        u'ugh',
        u'worth',
        u'ago',
        u'odd',
        u'among',
        u'Until',
        u'albeit',
        u'Through',
        u'annoys',
        u'enough',
        u'Behind',
        u'appreciate',
        u'>',
        u'Ok',
        u'That',
        u'tell',
        u'anyways',


        u'usefull',
        u'south',
        u'scan',
        u'save',
        u"'people",
        u'spiritual',
        u'starting',
        u'generous',
        u'pop',
        u'yadda',
        u'limited',
        u'opinionated',
        u'explanatory',
        u'failed',
        u'adorable',
        u'oblivious',
        u'worded',
        u'impersonal',
        u'inside',
        u'Couple',
        u'un-airbrushed',
        u'deep-red',
        u'canned',
        u'gentle',
        u'lucky',
        u'two-sided',
        u'opaque',
        u'latter',
        u"'friend",
        u'desperate',
        u'non-religious',
        u'metaphysics',
        u'absent',
        u'flighty/whimsically-flirtatious',
        u'standardized',
        u'arbitrary',
        u'Political',
        u'stable',
        u'degree',
        u'prone',
        u'non-ISTJs',
        u'kind/respectful',
        u'eager',
        u'ever-ready',
        u'indebted',
        u'long-winded',
        u'semi-pointl

        u'intuitive',
        u'order',
        u'chaos',
        u'theory',
        u'introvert',
        u'grown',
        u'awhile',
        u'club',
        u'boat',
        u'company',
        u'illness',
        u'duty',
        u'data',
        u'forum',
        u'proclivity',
        u'introspection',
        u'extraversison',
        u'offline',
        u'theorem',
        u'concentration',
        u'knowledge',
        u'year..',
        u'thank',
        u'tower',
        u'note',
        u'intellectual/non',
        u'divide',
        u'make-a-character',
        u'option',
        u'female',
        u'method',
        u'execution',
        u'ninja',
        u'brainpower',
        u'slavery',
        u'gift',
        u'bookmark',
        u'boy',
        u'weight',
        u'culture',
        u'elitism',
        u'grammar',
        u'wisdom',
        u'phrase',
        u'deadpool',
        u'victory',
        u'pose',
        u'b',
        u'result',
        u'impatience',
 

        u'gosh',
        u'orc',
        u'sport',
        u'Malekith',
        u'Teclis',
        u'Tyrion',
        u'goat',
        u'heathen',
        u'yo',
        u'perfectionist',
        u'saying',
        u'whilst',
        u'Girlfriend',
        u'Type',
        u'edit',
        u'Possession',
        u'Master',
        u'misunderstand',
        u'dingus',
        u'faith',
        u'rabbi',
        u'wang',
        u'XNFP',
        u'it/see',
        u'pragmatism',
        u'skepticism',
        u'adage',
        u'edge',
        u'enterprise',
        u'interpretation',
        u'destruction',
        u'objectivity',
        u'favourite',
        u'thesis',
        u'XNTP',
        u'Napoleon',
        u'correlation',
        u'build',
        u'masculinity',
        u'rationality',
        u'whiny',
        u'nu',
        u'burtmun',
        u'graduate',
        u'whatsapp',
        u'feature',
        u'ignore',
        u'crying',
        u'Release',
        u'ban',
    

...

         u'Lillith',
         u'Cry',
         u'ThoughtProcess',
         u'PT',
         u'BC/Vancouver',
         u'Island',
         u'Bedouin',
         u'Path',
         u'Beckoning',
         u'CERN',
         u'Haruka',
         u'Nanase-',
         u'Haru',
         u'Makoto',
         u'Tachibana-',
         u'Rin',
         u'Matsuoko-',
         u'Frisk',
         u'Dominant',
         u'Bubbles',
         u'Dom',
         u'Aux',
         u'Boundary',
         u'Kuriyama',
         u'Akihito',
         u'Kanbara',
         u'Mitsuki',
         u'Nase',
         u'Hiroomi',
         u'xNFJ',
         u'ESFx',
         u'IxTP',
         u'xNFP',
         u'Stereotypical',
         u'Worst',
         u'Ji',
         u'Additional',
         u'Thinks',
         u'Ambiguous',
         u'Ushijima',
         u'Initially',
         u'Uhm',
         u'Short',
         u'Gotterdammerung',
         u'MENTION=1223',
         u'Ymir',
         u'Shingeki',
         u'Kyojin',
         u

         u'creatures',
         u'causes',
         u'handers',
         u'hands',
         u'behaviours',
         u'techniques',
         u'Meetings',
         u'walls',
         u'details',
         u"'Listen",
         u'streets',
         u'squares',
         u'lips',
         u'ambitions',
         u'sentiments',
         u'tips',
         u'manuals',
         u'bores',
         u'heads',
         u'gatherings',
         u'objectives',
         u'devices',
         u'grades',
         u'notes',
         u'channels',
         u'commercials',
         u'commentators',
         u'favorites',
         u'smudges',
         u'headsets',
         u'phones',
         u'puns',
         u'Files',
         u'-Chronicles',
         u'metaphors',
         u'meanings',
         u'tends',
         u'mistakes',
         u'conditions',
         u'cats',
         u'extremes',
         u'chairs',
         u'mad',
         u'cultures',
         u'vapors',
         u'magistrates',
         u'Cheers',

        u'repeatedly',
        u'names',
        u'defensively',
        u'sincerely',
        u'authentically',
        u'Lovely',
        u'unlikely',
        u'reaallly',
        u'berserk',
        u'freshly',
        u'type',
        u'xNTP',
        u'ExFP',
        u'Firstly',
        u'unconsciuosly'],
 'RBR': [u'better',
         u'more',
         u'less',
         u'faster',
         u'further',
         u'MORE',
         u'easier',
         u'later',
         u'longer',
         u'Less',
         u'Better',
         u'More',
         u'wittier',
         u'earlier'],
 'RBS': [u'most', u'Best', u'best', u'^^SO', u'Most'],
 'RP': [u'out',
        u'up',
        u'on/off',
        u'off',
        u'down',
        u'forth',
        u'back',
        u'XP',
        u'over',
        u'around',
        u'apps',
        u'round',
        u'outweighs',
        u'away',
        u'apart',
        u'\u201c',
        u'Back'],
 'SYM': [u'YouTube',
         u'misunderstood',
         u'jok

         u'revised',
         u'reputed',
         u'Psychopaths',
         u'became',
         u'taught',
         u'waited',
         u'punched',
         u'opened',
         u'cried',
         u'knocked',
         u'chose',
         u'effected',
         u'cut',
         u'Introverted',
         u'signed',
         u'grew',
         u'brought',
         u'presented',
         u'bounced',
         u'reacted',
         u'moreso',
         u'karaoke',
         u'explored',
         u'shit',
         u'studied',
         u'lied',
         u'surprised',
         u'divulged',
         u'recommended',
         u'fought',
         u'deserved',
         u'w/',
         u'initiated',
         u'mellowed',
         u'relied',
         u'enamoured',
         u'tested',
         u'remembered',
         u'wavered',
         u'asked',
         u'walked',
         u'rocked',
         u'yelled',
         u'determined',
         u'meepvedev',
         u'phobic',
         u'bet',
         u'majored',


         u'detected',
         u'reminded',
         u'encountered',
         u'evidenced',
         u'acquainted',
         u'legalized',
         u'coupled',
         u'hatred',
         u'Started',
         u'commissioned',
         u'knocked',
         u'accomplished',
         u'epic',
         u'accelerated',
         u'passed',
         u'laid',
         u'procrastinated',
         u'impacted',
         u'inclined',
         u'suffered',
         u'pursued',
         u'overtaken',
         u'expressed',
         u'pushed',
         u'managed',
         u'improved',
         u'smoked',
         u'INFJ',
         u'motvated',
         u'calmed',
         u'dower',
         u'sounded',
         u'Souled',
         u'pondered',
         u'replaced',
         u'OrangeAppled',
         u'conveyed',
         u'dismayed',
         u'capped',
         u'lost/abandoned',
         u'divorced',
         u'purchased',
         u'repulsed',
         u'scattered'],
 'VBP': [u'are',
         u'

In [ ]:
len(wordcatdict.keys())

In [ ]:
columnname = [col for col in wordcatdict.keys()]
print columnname

In [ ]:
for col in columnname:
    newlist=[]
    for line in mbti['tagged_words'][0]:
        newlist.append(len([x for x in line if x[1]==col]))
    print "For "+col+","
    print "Sum = ", np.sum(newlist)
    print "Variance =", np.var(newlist)
    print "Mean =",np.mean(newlist)
    print "Median =",np.median(newlist)
    print "Standard Deviation =",np.std(newlist)

So...

Variance and median looks useless... since some words are not used very often. We are dealing with posts of varying lengths and they may tend to contain a wide range, from little/no words to a lengthy discussion.

If I run my model with a chatbot it might or might not succeed, since I reckon that text messaging would tend to be shorter.

Some adjustments need to be made

In [ ]:
#Let us visualise one of the word types across a spectrum
def plot_wordtype(ind, wtype):
    newlist=[]
    for line in mbti['tagged_words'][ind]:
        newlist.append(len([x for x in line if x[1]==wtype]))
        
    newlist.sort()
    plt.figure(figsize=(10,5))
    plt.plot(np.arange(1, len(newlist)+1), newlist)
    plt.xlabel("Index")
    plt.ylabel("Frequency of "+wtype+" word")
    plt.title("Frequency plot of words of type "+wtype+" for MBTI type "+mbti['type'][ind])

In [ ]:
# plot_wordtype(145, 'RB')

In addition to simply counting all that occurs, I think it would be good to kinda 'summarize' them

Like for example the nouns come in many forms, like NN, NNP, NNS, NNPS - but we can group all together, for instance.

Such distinction is useful for word generation I guess but not that useful for purpose of my use.

Lets see... Now I shall do the following:

Tag	| Meaning	|English Examples
----|-----------|------------------
ADJ	|adjective	|new, good, high, special, big, local
ADP	|adposition	|on, of, at, with, by, into, under
ADV	|adverb	|really, already, still, early, now
CONJ |conjunction	|and, or, but, if, while, although
DET	|determiner, article	|the, a, some, most, every, no, which
NOUN|	noun	|year, home, costs, time, Africa
NUM	|numeral	|twenty-four, fourth, 1991, 14:24
PRT	|particle	|at, on, out, over per, that, up, with
PRON|	pronoun	|he, their, her, its, my, I, us
VERB|	verb	|is, say, told, given, playing, would
.	|punctuation marks	|. , ; !
X	|other	|ersatz, esprit, dunno, gr8, univeristy

But I will still keep the original columns heh

In [18]:
#Create columns to get the mean and std for each POS tagging for each row
def pos_cat(x, wordie):
    return [len([y for y in line if y[1] == wordie])for line in x]
    

# for col in columnname:
#     mbti['POS_'+col+'_mean'] = mbti['tagged_words'].apply(lambda x: np.mean(pos_cat(x, col)))
#     mbti['POS_'+col+'_std'] = mbti['tagged_words'].apply(lambda x: np.std(pos_cat(x, col)))
# mbti.head()

In [19]:
#For reference ;)
lollie = """
CC Coordinating conjunction *
CD Cardinal number *
DT Determiner *
EX Existential there *
FW Foreign word *
IN Preposition or subordinating conjunction *
JJ Adjective *
JJR Adjective, comparative *
JJS Adjective, superlative *
LS List item marker *
MD Modal *
NN Noun, singular or mass *
NNS Noun, plural *
NNP Proper noun, singular *
NNPS Proper noun, plural *
PDT Predeterminer *
POS Possessive ending ?
PRP Personal pronoun *
PRP$ Possessive pronoun *
RB Adverb *
RBR Adverb, comparative *
RBS Adverb, superlative *
RP Particle *
SYM Symbol ?
TO to *
UH Interjection *
VB Verb, base form *
VBD Verb, past tense
VBG Verb, gerund or present participle *
VBN Verb, past participle *
VBP Verb, non­3rd person singular present *
VBZ Verb, 3rd person singular present *
WDT Wh­determiner *
WP Wh­pronoun *
WP$ Possessive wh­pronoun *
WRB Wh­adverb *
"""

#Lets make a dictionary
convtag_dict={'ADJ':['JJ','JJR','JJS'], 'ADP':['EX','TO'], 'ADV':['RB','RBR','RBS','WRB'], 'CONJ':['CC','IN'],'DET':['DT','PDT','WDT'],
              'NOUN':['NN','NNS','NNP','NNPS'], 'NUM':['CD'],'PRT':['RP'],'PRON':['PRP','PRP$','WP','WP$'],
              'VERB':['MD','VB','VBD','VBG','VBN','VBP','VBZ'],'.':['#','$',"''",'(',')',',','.',':'],'X':['FW','LS','UH']}

In [31]:
def bigcol_tag(x, col):
    newlist = [len([y for y in line if y[1] in convtag_dict[col]])for line in x]       
    return newlist

for col in convtag_dict.keys():  
    mbti['BIGPOS_'+col+'_med'] = mbti['tagged_words'].apply(lambda x: np.median(bigcol_tag(x, col)))
    mbti['BIGPOS_'+col+'_std']= mbti['tagged_words'].apply(lambda x: np.std(bigcol_tag(x, col)))

In [17]:
mbti['n_mbti_self'] = mbti.apply(lambda x: 1 if x['type']==x['mbti_ref_most'] else 0, axis=1)

Pickle

In [ ]:
mbti.to_pickle('mbti_ver2.pickle')

# \**Checkpoint \**

In [4]:
mbti = pd.read_pickle('mbti_ver2.pickle')

In [5]:
#Take out irrelevant columns
mbti_w = mbti[['words_only','image','video_title','otherlink','emoticons','ennea','tagged_words']]
mbti_n = mbti.drop(['type','image','video_link','video_title',
                    'otherlink','emoticons','hashtag','mbti_ref','mbti_ref_most','ennea','tagged_words','words_refine',
                   'tagged_words'], axis=1)

In [6]:
mbti_n.columns

Index([u'words_only', u'n_video', u'n_links', u'n_image', u'n_otherlink',
       u'emoticon_count', u'mention_count', u'hashtag_count',
       u'mbti_ref_count', u'ennea_count',
       ...
       u'BIGPOS_NOUN_std', u'BIGPOS_PRON_med', u'BIGPOS_PRON_std',
       u'BIGPOS_._med', u'BIGPOS_._std', u'BIGPOS_PRT_med', u'BIGPOS_PRT_std',
       u'BIGPOS_NUM_med', u'BIGPOS_NUM_std', u'n_mbti_self'],
      dtype='object', length=141)

In [7]:
mbti_n['words_only'] = mbti_n['words_only'].apply(lambda x: ' '.join(x))

# Term Frequency - Inverse Document Frequency

From here on I will start to split the dataset by the following:

1. Duplicate the set by 4, with 4 types of ys using is_E, is_S, is_T and is_J
2. Fit the model once with the full set
3. Do a stratified split for all 4 sets, so I will have 4 sets of X_train and 4 sets of X_test
4. Transform all of them
5. Using a chi-squared test, try to scale down the dimension separately

So from here on actually we might start to see how each feature may be more effective in one model than another. From here on each model may start to 'deviate' from one another



In [8]:
#We love sklearn <3 <3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression

In [9]:
mbti_n['word_char_ratio'] = mbti_n['word_char_ratio'].fillna(np.median(mbti_n[~mbti_n['word_char_ratio'].isnull()]['word_char_ratio']))

In [10]:
y_E = mbti['is_E']
y_S = mbti['is_S']
y_T = mbti['is_T']
y_J = mbti['is_J']
X = mbti_n

class Thipe(object):
    
    def __init__(self, X, y, stan=True, rand=42):
        self.stan=stan
        self.X=X
        self.y=y
        self.random_state = rand
        self.columns=X.columns
        self.X_train=None
        self.y_train=None
        #I will be porting some features to other classes sooooo...
        self.tfidf_list=[]
        self.tsvd_list=[]
        self.ss=None
        self.mas=None
        self.ch2=None
        
    def baseline_acc(self):
        baseline = max(self.y.value_counts()[0], self.y.value_counts()[1]) / float(self.y.value_counts().sum())
        return baseline * 100,'\%'
        
        
    def trainy(self, testy=0.2):
        """
        I'll do the following here:
        1. Do train test split
        2. Convert X_train and X_test to DataFrame (to delete column later plus other purposes)
        3. Do tfidf using train section, use the model and fit the X_train and X_test (then can delete the wordchunk column)
        4. If StandardScaler, scale the training and test data. (Default = True)
        5. To prepare data for chi2 reduction we need to scale everything to above 0, so MinMaxScaler
        """
               
        #This is perhaps the main reason why this step is embedded in a class
        #Because the stratification would be different, everything would be different already, like the tfidf vocab for example
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, random_state=self.random_state,
                                                                      test_size=testy, stratify=self.y)
        self.y_train=y_train
        self.y_test=y_test
        
        X_train = pd.DataFrame(X_train, columns=self.columns)
        X_test = pd.DataFrame(X_test, columns=self.columns)
        df_train = pd.DataFrame()
        df_test = pd.DataFrame()
        for i in np.arange(1,4):
            tfidf = TfidfVectorizer(stop_words='english',ngram_range=(i,i), decode_error='replace', max_features=10000)
            Xword_train = tfidf.fit_transform(X_train['words_only'])
            Xword_test = tfidf.transform(X_test['words_only'])
            
            #We need to reduce the size of the tfidf trained matrix first
            #But after running TruncatedSVD we cannot see the words specifically alr so too bad...
            tsvd = TruncatedSVD(n_components=500, algorithm='arpack', random_state=self.random_state)
            Xwordie_train = tsvd.fit_transform(Xword_train)
            Xwordie_test = tsvd.transform(Xword_test)
            Xwordie_train_df = pd.DataFrame(Xwordie_train,
                                            columns=[str(i)+'_'+str(b) for b in np.arange(1,Xwordie_train.shape[1]+1)])
            Xwordie_test_df = pd.DataFrame(Xwordie_test,
                                           columns=[str(i)+'_'+str(b) for b in np.arange(1,Xwordie_test.shape[1]+1)])
            df_train = pd.concat([df_train,Xwordie_train_df], axis=1)
            df_test = pd.concat([df_test,Xwordie_test_df], axis=1)
            self.tfidf_list.append(tfidf)
            self.tsvd_list.append(tsvd)
        
        X_train.drop(['words_only'],axis=1,inplace=True)
        X_test.drop(['words_only'],axis=1,inplace=True)
        columns = X_train.columns
        
        #Standardization step
        if self.stan:
            X = self.X.drop(['words_only'], axis=1)
            ss = StandardScaler().fit(X)
            X_train = ss.transform(X_train)
            X_test = ss.transform(X_test)
            X_train = pd.DataFrame(X_train, columns=columns)
            X_test = pd.DataFrame(X_test, columns=columns)
            self.ss = ss
            
        #Join step
        X_train = X_train.join(df_train)
        X_test = X_test.join(df_test)
        columnie = X_train.columns
        
        #Scale again to between 0 and 1
        combined_X = pd.concat([X_train,X_test],axis=0)
        mms = MinMaxScaler().fit(combined_X)
        X_train = pd.DataFrame(mms.transform(X_train), columns=columnie)
        X_test = pd.DataFrame(mms.transform(X_test), columns=columnie)
        self.X_train = X_train
        self.X_test = X_test
        self.mms = mms
        
        return X_train, X_test, y_train, y_test
    
    def reducy(self, tipe = 'chi2', n_features=100):
        """
        Use chi2 to shrink down features
        """
        ch2 = SelectKBest(chi2, k=n_features)
        X_train = ch2.fit_transform(self.X_train, self.y_train)
        X_test = ch2.transform(self.X_test)
        self.X_train=X_train
        self.X_test=X_test
        return X_train, X_test

        
    def try_model(self, model, label):
        sexy = model
        baby = sexy.fit(self.X_train, self.y_train)
        print logr.score(self.X_test, self.y_test)
        yhat = logr.predict(X_test)
        print classification_report(y_test, yhat, target_names=label)
        
        
    def niceplot(self):
        pass
    

In [11]:
E = Thipe(X, y_E, rand=45)

In [12]:
# from tpot import TPOTClassifier
X_train, X_test, y_train, y_test = E.trainy()
X_train, X_test = E.reducy()

C:\Users\User\Anaconda2\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [13]:
# tpot = TPOTClassifier(generations=10, population_size=30, verbosity=2, scoring='f1')
# tpot.fit(X_train, y_train)
# print(tpot.score(X_test, y_test))
# tpot.export('tpot_E_try.py')

In [14]:
#ok yes it has reduced in shape
X_train.shape

(6940L, 100L)

We first look at how the model performs with logistic regression

In [15]:
logr = LogisticRegression()
model = logr.fit(X_train, y_train)
print logr.score(X_test, y_test)

1.0


This is a classification problem, so of course we cannot just look at the score alone

In [16]:
yhat = logr.predict(X_test)
print classification_report(y_test, yhat, target_names=['Introvert','Extrovert'])

             precision    recall  f1-score   support

  Introvert       1.00      1.00      1.00      1335
  Extrovert       1.00      1.00      1.00       400

avg / total       1.00      1.00      1.00      1735



Looking at the score honestly I abit scared now...

In [17]:
print confusion_matrix(y_test, yhat)
#Wow

[[1335    0]
 [   0  400]]


### I guess I need to appeal for help again, is there really such a thing as perfection? Or am I just lucky? Or maybe...

Don't forget, other predictors!

In [22]:
tpot = TPOTClassifier(generations=10, population_size=30, verbosity=2, scoring='f1')

In [23]:
S = Thipe(X, y_S)
X_train, X_test, y_train, y_test = S.trainy()
X_train, X_test = S.reducy()
S.try_model(model=LogisticRegression(), label=['Intuitive','Sensing'])

0.585014409222
             precision    recall  f1-score   support

  Intuitive       0.86      0.62      0.72      1496
    Sensing       0.14      0.39      0.21       239

avg / total       0.76      0.59      0.65      1735



In [24]:
tpot.fit(S.X_train, S.y_train)
print(tpot.score(S.X_test, S.y_test))
tpot.export('tpot_S_try.py')

Generation 1 - Current best internal CV score: 1.0


Generation 2 - Current best internal CV score: 1.0


Generation 3 - Current best internal CV score: 1.0


Generation 4 - Current best internal CV score: 1.0


Generation 5 - Current best internal CV score: 1.0


...

Generation 8 - Current best internal CV score: 1.0


Generation 9 - Current best internal CV score: 1.0


Generation 10 - Current best internal CV score: 1.0



Best pipeline: RandomForestClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.6, min_samples_leaf=4, min_samples_split=17, n_estimators=100)
1.0


True

In [25]:
T = Thipe(X, y_T)
X_train, X_test, y_train, y_test = T.trainy()
X_train, X_test = T.reducy()
T.try_model(model=LogisticRegression(), label=['Feeling','Thinking'])

0.468011527378
             precision    recall  f1-score   support

    Feeling       0.54      0.12      0.19       939
   Thinking       0.46      0.88      0.60       796

avg / total       0.50      0.47      0.38      1735



In [26]:
tpot.fit(T.X_train, T.y_train)
print(tpot.score(T.X_test, T.y_test))
tpot.export('tpot_T_try.py')

Generation 1 - Current best internal CV score: 1.0


Generation 2 - Current best internal CV score: 1.0


Generation 3 - Current best internal CV score: 1.0


Generation 4 - Current best internal CV score: 1.0


Generation 5 - Current best internal CV score: 1.0


...

Generation 8 - Current best internal CV score: 1.0


Generation 9 - Current best internal CV score: 1.0


Generation 10 - Current best internal CV score: 1.0



Best pipeline: RandomForestClassifier(input_matrix, bootstrap=True, criterion=entropy, max_features=0.85, min_samples_leaf=3, min_samples_split=8, n_estimators=100)
1.0


True

In [27]:
J = Thipe(X, y_J)
X_train, X_test, y_train, y_test = J.trainy()
X_train, X_test = J.reducy()
J.try_model(model=LogisticRegression(), label=['Perceiving','Judging'])

0.594812680115
             precision    recall  f1-score   support

 Perceiving       0.61      0.94      0.74      1048
    Judging       0.42      0.06      0.11       687

avg / total       0.53      0.59      0.49      1735



In [28]:
tpot.fit(J.X_train, J.y_train)
print(tpot.score(J.X_test, J.y_test))
tpot.export('tpot_J_try.py')

Generation 1 - Current best internal CV score: 1.0


Generation 2 - Current best internal CV score: 1.0


Generation 3 - Current best internal CV score: 1.0


Generation 4 - Current best internal CV score: 1.0


Generation 5 - Current best internal CV score: 1.0


...

Generation 8 - Current best internal CV score: 1.0


Generation 9 - Current best internal CV score: 1.0


Generation 10 - Current best internal CV score: 1.0



Best pipeline: GaussianNB(input_matrix)
1.0


True

## Now comes the half exciting part: Processing brand new data!

After training this data, of course what we really want to do with it is to use the trained model to predict fresh new input.
The class Thipe was used to train the model according to the 4 types, now we should have 4 instances to use.

For each instance, we have:
- 3 x tfidf plus 3 x tsvd
- StandardScaler
- MinMaxScaler
- SelectKBest(Using Chi2)
- Logistic Regression (or whichever model you like)

Process and process, until like from pig until Premium Sausage and Bacon flavoured cup noodles with Tonkotsu broth

In [ ]:
class NewBerd(Thipe):
    
    def __init__(self, ):
        pass
    
    #I'll figure out this shit later
    